In [2]:
#pip install gradio

In [235]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import datetime
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import os
import shutil

In [149]:
def get_rank_participant(bib_number):
    finishers = pd.read_csv('./data/finishers.csv', index_col=0)
    try:
        rank = f"Congratulations ! You are {finishers[finishers['bib_number']==bib_number].index.array[0] + 1}th out of {finishers.shape[0]} finishers."
    except: 
        rank = "Sorry, you are not registered as a finisher."
    return rank

In [148]:
print(get_rank_participant(5))

Sorry, you are not registered as a finisher.


In [182]:
def boxplot_participant(bib_number):
    time_by_5 = pd.read_csv('./data/times_cutoff_5km.csv', index_col=0)

    plt.figure(figsize=(16,8))
    fig = sns.boxplot(time_by_5.iloc[:,1:]/60)
    plt.xlabel('Covered distance');
    plt.ylabel('Race time for the latter 5 km run  [min]');

    medians = time_by_5.iloc[:,1:].median()/60
    offset_vert = medians * 0.01
    cpt = 0
    for xtick in fig.get_xticks():
        fig.text(xtick,medians.iloc[xtick] + offset_vert.iloc[xtick],np.round(medians.iloc[xtick],2), horizontalalignment='center',size='x-small',color='b',weight='semibold')
        cpt += 1
        if cpt==1:
            fig.plot(xtick,time_by_5[time_by_5['bib_number']==bib_number].iloc[:,xtick+1]/60, 'ro', label= f'Bib number: {bib_number}')
        else:
            fig.plot(xtick,time_by_5[time_by_5['bib_number']==bib_number].iloc[:,xtick+1]/60, 'ro')
    plt.legend();
    if not os.path.exists("./data/temp_img"):
        os.makedirs("./data/temp_img")
    plt.savefig(f"./data/temp_img/{bib_number}.png")
    return f"./data/temp_img/{bib_number}.png"  

In [208]:
def race_data_participant(bib_number):
    finishers = pd.read_csv('./data/finishers.csv', index_col=0)
    headers=["last_name", "first_name", "half_1st","half_second","time_total_netto","time_total_brutto"]
    try:
        out = finishers.loc[finishers['bib_number']==bib_number, headers]
        out['half_1st'] = out['half_1st'].apply(lambda row : str(datetime.timedelta(seconds = row)))
        out['half_second'] = out['half_second'].apply(lambda row : str(datetime.timedelta(seconds = row)))
        out['time_total_netto'] = out['time_total_netto'].apply(lambda row : str(datetime.timedelta(seconds = row)))
        out['time_total_brutto'] = out['time_total_brutto'].apply(lambda row : str(datetime.timedelta(seconds = row)))
    except: 
        out = pd.DataFrame()
    return out

In [209]:
race_data_participant(13440)

,last_name,first_name,half_1st,half_second,time_total_netto,time_total_brutto
17106,A.,Y.,3:28:11,4:03:09,7:31:20,7:33:12


In [259]:
def pie_plot():
    df = pd.read_csv('./data/cleaned_data_marathon_all_participants.csv', index_col=0)
    finishers = pd.read_csv('./data/finishers.csv', index_col=0)
    df = pd.DataFrame(data={'Participants': [finishers.shape[0], df.shape[0]-finishers.shape[0]]}, index=['Finishers', 'Not finishers'])
    colors = sns.color_palette('bright')[0:5]
    fig = px.pie(df, values='Participants', names=df.index);
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig

In [260]:
pie_plot()

In [262]:
demo = gr.Blocks()

title=(
"""
<center> 

<h1> Marathon pour tous, 10 août 2024 👟 </h1>
<b> Site non officiel d'analyse des résultats 🏅<b>

</center>
"""
)

def task1(bib_number):
    rank = get_rank_participant(bib_number)
    return rank

def task1bis(bib_number):
    out = race_data_participant(bib_number)
    return out

def task2(bib_number):
    return boxplot_participant(bib_number)

def delete_directory():
    shutil.rmtree("./data/temp_img")

with demo:
    with gr.Row():
          gr.HTML(title)  
      
    with gr.Row():
        gr.Markdown("""Le site officiel avec les informations relatives à cet événement est le suivant : https://marathonpourtous.paris2024.org/fr/l-evenement/marathon-pour-tous""")
        
    with gr.Tabs():
        with gr.TabItem("Your ranking position"):
            with gr.Row():
                bib_rank_input = gr.Number()
                text_output = gr.Textbox()
            rank_button = gr.Button("Get your rank position from your bib number")
            results_button = gr.Button("Get your race data from your bib number")
            results_output = gr.Dataframe(
                headers=["last_name", "first_name", "half_1st","half_second","time_total_netto","time_total_brutto"],
                datatype=["str","str", "str", "str","str","str"],
                row_count=1,
                col_count=(6, "fixed"),
                label="Your race results")
            
        with gr.TabItem("Analysis of 5km sections"):
            with gr.Row():
                bib_5km_input = gr.Number()
            image_button = gr.Button("Analysis of split times - 5 km sections")
            gr.Markdown(""" The coloured boxes show the distribution of the times taken by all the finishers to cover the latter 5km section  """)
            image_output = gr.Image()
            gr.Markdown("""For each 5 km section, 50% of runners :\n
                        - have a time within the range indicated by the coloured box\n
                        - have a time below the median (horizontal line in the coloured box)\n
                        - have a time above the median (horizontal line in the coloured box)\n
                        The value in blue is the median, in minutes.\n
                        The points in red correspond to the time achieved by th runner associated with the given bib number.""")
        
        with gr.TabItem("A few statistics"):
            with gr.Row():
                gr.Plot(pie_plot())
                gr.Image('./data/img/net_total_time_distribution.png')

    rank_button.click(task1, inputs=bib_rank_input, outputs=text_output)
    results_button.click(task1bis, inputs=bib_rank_input, outputs=results_output)
    image_button.click(task2, inputs=bib_5km_input, outputs=image_output)
    if os.path.exists("./data/temp_img"):
        demo.unload(delete_directory)

# Run the interface
demo.launch()

Running on local URL:  http://127.0.0.1:7924

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\dcons\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dcons\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dcons\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dcons\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dcons\AppData\Local\P